### Day 4

Here we have a series of bingo boards and a sequence of numbers called. We're tasked with determining which board won, when, and what the uncalled numbers in that board are

In [39]:
import pandas as pd
import numpy as np
from functools import reduce

with(open('d4.txt')) as text:
    puzzle_input = text.readlines()

numbers_called = puzzle_input[0].replace("\n", '')
numbers_called = numbers_called.split(',')
numbers_called = [int(x) for x in numbers_called]
numbers_called[:5]

[92, 12, 94, 64, 14]

In [40]:
# Now get the bingo boards
boards = puzzle_input[2:]
boards = [x.replace('\n', '') for x in boards]
boards = list(map(lambda x: [x for x in x.split(' ') if len(x) > 0], boards))
boards = [x for x in boards if len(x) == 5]

boards[0:5]

[['60', '79', '46', '9', '58'],
 ['97', '81', '6', '94', '84'],
 ['38', '40', '17', '61', '29'],
 ['11', '28', '0', '91', '15'],
 ['24', '77', '34', '59', '36']]

In [41]:
def make_board(start):
    return np.array(boards[start:(start+5)], dtype = 'int')

boards = list(
    map(
        make_board, 
        list(range(0, len(boards), 5))
    )
)

# Save a copy for testing
boards_save = boards.copy()


In [42]:
boards[0]

array([[60, 79, 46,  9, 58],
       [97, 81,  6, 94, 84],
       [38, 40, 17, 61, 29],
       [11, 28,  0, 91, 15],
       [24, 77, 34, 59, 36]])

I think this will be easiest if we just replace values with an indicator value. Part 2 might not like having all that information lost but whatever 

Need a function to check if the board is done

In [43]:
def all_marked(x):
    return all(x == -1)

def bingo(x):
    row_bingo = np.apply_along_axis(all_marked, 0, x).any()
    col_bingo = np.apply_along_axis(all_marked, 1, x).any()
    return (row_bingo or col_bingo)

Now we wanna just loop through all the numbers called in order, modify the boards, and then check if they're done


In [44]:
done = False
for number in numbers_called:
    
    # Replace number in boards
    for i, board in enumerate(boards):
        boards[i] = np.where(
            boards[i] == number,
            -1,
            boards[i]       
        )
        # Check for a bingo
        if bingo(boards[i]):
            winning_board = boards[i]
            done = True
            break
    
    if done:
        break

number*np.where(
    winning_board == -1,
    0,
    winning_board
).sum()

33462

### Part 2

Now we have to run through and find the last board to get a bingo. Basically the same except now we delete boards when they bingo instead of stopping

In [59]:
boards = boards_save.copy()

for number in numbers_called:
    
    # list of boards to remove
    removals = []
    
    # Replace number in boards
    for i, board in enumerate(boards):
        boards[i] = np.where(
            boards[i] == number,
            -1,
            boards[i]       
        )
        # Check for a bingo
        if bingo(boards[i]):
            removals.append(i)
    
    if len(boards) == 1:
        worst_board = boards[0]
        break 
    
    # I don't really know how to remove items from a list by position in
    # a better way. The index changes as you remove items:
    offset = 0
    for removal in removals:
        del boards[removal + offset]
        offset -= 1
    
number*np.where(
    worst_board == -1,
    0,
    worst_board
).sum()   
    

30070

In [58]:
boards

[array([[-1, -1, 58, -1, -1],
        [-1, -1, -1, 43, 41],
        [-1, 80, -1, -1, -1],
        [-1, -1, 61, -1, -1],
        [-1, 27, -1, -1, -1]])]

In [51]:
boards.remove(removals[0])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [53]:
removals[0] in boards


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()